In [ ]:
!pip install --upgrade imutils

In [ ]:
#set the matplotlib backend
import matplotlib
matplotlib.use("agg")
# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import pickle
import cv2
import os

In [ ]:
# construct the argument parser and parse the arguments
ap = argparse.ArgumentParser()
ap.add_argument("-d", "--dataset", required=True,
                help="path to input dataset of images")
ap.add_argument("-m", "--model", required=True,
                help="path to output trained model")
ap.add_argument("-l", "--label-bin", required=True,
                help="path to output label binarizer")
ap.add_argument("-p", "--plot", required=True,
                help="path to output accuracy/loss plot")
args = vars(ap.parse_args())

In [ ]:
#initialize data and labels
print("[INFO] loading images..")
data = []
labels = []

#grab the image paths and randmly shuffle them
#imagepaths = sorted(list(paths.list_images(args["dataset"])))
imagePaths = sorted(list(paths.list_images("/Users/trinakarmakar/downloads/keras-tutorial/animals/")))

random.seed(42)
random.shuffle(imagePaths)

#loop over the inout images
for imagePath in imagePaths:
    #load the image resize the inage to be 32x32 pixels (ignoring aspect ratio),
    #flatten the image unto 32X32#3 = 3027 pixel image
    #into a list, and store the image into a data list
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (32,32)).flatten()
    data.append(image)
    
    #extract the class label from the image and update the 
    #labels list
    label = imagePath.split(os.path.sep)[-2]
    labels.append(label)

In [ ]:
#scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

In [ ]:
#partition the data into training and testing splits using 75% of
# the data for training and the remaining 25% for testing
(X_train, X_test, y_train, y_test) = train_test_split(data, labels, test_size=0.25, random_state=42)

In [ ]:
#Convert the labels from integers to vectors (for 2-class binary
#classification you should use Keras 'to_categorical' function
#instead as the scikit-learn's labelBinarizer will not return a
#a vector)

lb = LabelBinarizer()
y_train = lb.fit_transform(y_train)
y_test = lb.transform(y_test)

In [ ]:
#Define teh 3072-1074-512-3 architecture using keras
model = Sequential()
model.add(Dense(1024, input_shape=(3072,), activation='sigmoid'))
model.add(Dense(512, activation='sigmoid'))
model.add(Dense(len(lb.classes_), activation='softmax'))
model.summary()

In [ ]:
#Intialize our initial learning rate and # of epochs to train for
INIT_LR = 0.01
EPOCHS=75

#Compiling the model using SGD as our optimizer and categotical
# cross-entropy loss (you'll want to use binary_crossentropy)
#for 2 class classification)

print("[INFO] training network...")
opt = SGD(lr=INIT_LR)
model.compile(loss="categorical_crossentropy", optimizer=opt,
             metrics=["accuracy"])

In [ ]:
#train the neural network
history = model.fit(X_train, y_train, 
                    validation_data=(X_test, y_test),
                   epochs=EPOCHS, batch_size=32)

In [ ]:
#Evaluate the network
print("[INFO] evaluating the network...")
predictions = model.predict(X_test, batch_size=32)
print(classification_report(y_test.argmax(axis=1),
                           predictions.argmax(axis=1), 
                            target_names=lb.classes_))

#Plot the training loss and accuracy
N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, history.history["loss"], label="train_loss")
plt.plot(N, history.history["val_loss"], label="val_loss")
plt.plot(N, history.history["acc"], label="train_acc")
plt.plot(N, history.history["val_acc"], label="val_acc")
plt.plot("Training loss and accuracy (simlple NN)")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.show()
#plt.savefig(args["plot"])
plt.savefig("../downloads/acc_report.png")

In [ ]:
#Save the model and label binarizer to disk
print("[INFO] serializing network and label binarizer...")
#model.save(args["model"])
model.save("../downloads/image_classification.model")
#f = open(args["label_bin"], "wb")
f = open("../downloads/image_classification_lb.pickle", "wb")
f.write(pickle.dumps(lb))
f.close()

In [ ]:
#Predictions/Inference
from keras.models import load_model
import argparse
import pickle
import cv2

#construct the argument parser and parse the argumemts
ap = argparse.ArgumentParser()
ap.add_argument("-i", "--image", required=True,
                help="path to input image we are going to classify")
ap.add_argument("-m", "--model", required=True,
                help="path to trained Keras model")
ap.add_argument("-l", "--label-bin", required=True,
                help="path to label binarizer")
ap.add_argument("-w", "--width", type=int, default=28,
                help="target spatial dimension width")
ap.add_argument("-e", "--height", type=int, default=28,
                help="target spatial dimension height")
ap.add_argument("-f", "--flatten", type=int, default=-1,
                help="whether or not we should flatten the image")
args = vars(ap.parse_args())

In [ ]:
#load the input image and resize it to the target spatial dimensions
#image = cv2.imread(args["image"])
image = cv2.imread("../downloads/cat.jpg")
output = image.copy()
#image = cv2.resize(image, (args["width"], args["height"]))
image = cv2.resize(image, (32, 32))

In [ ]:
#check to see if we should flatten the image and add a batch
#dimension
flatten = 1
#if args["flatten"] > 0:
if flatten > 0:
    image = image.flatten()
    image = image.reshape(1, image.shape[0])
    
#otherwise, we must be working with a CNN, --don't flatten the
# image simply add the batch dimension
else:
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))

In [ ]:
#load the model and labelbinarizer
print("[INFO] loading network and label binarizer...")
#model = load_model(args["model"])
model = load_model("../downloads/image_classification.model")
#lb = pickle.loads(open(args["label_bin"], "rb").read())
lb = pickle.loads(open("../downloads/image_classification_lb.pickle", "rb").read())
#make a prediction on the image
preds = model.predict(image)

#find the class label index with the largets corresponding
#probability
pred_idx = preds.argmax(axis=1)[0]
label = lb.classes_[pred_idx]

In [ ]:
preds

In [ ]:
#draw the class clabel + probability on the output image
text = "{}: {:.2f}%".format(label, preds[0][pred_idx] * 100)
cv2.putText(output, text, (10,30), cv2.FONT_HERSHEY_SIMPLEX, 0.7,
           (0, 0, 255), 2)

#SHOW THE OUTPUT IMAGE
cv2.imshow("Image", output)
cv2.waitKey(0)

Admittedly, using a standard feedforward neural network to classify images is not a wise choice.

Instead, we should leverage Convolutional Neural Networks (CNNs) which are designed to operate over the raw pixel intensities of images and learn discriminating filters that can be used to classify images with high accuracy.

The model we’ll be discussing here today is a smaller variant of VGGNet which I have named “SmallVGGNet”.

VGGNet-like models share two common characteristics:

Only 3×3 convolutions are used
Convolution layers are stacked on top of each other deeper in the network architecture prior to applying a destructive pooling operation
Let’s go ahead and implement SmallVGGNet now.

In [ ]:
#Import the necessary packages
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K

In [ ]:
class SmallVGGNet:
    @staticmethod
    def build_model(width, height, depth, classes):
        #Initialize the model along with the input shape to be
        # "channels last" and the channels dimension itself.
        model = Sequential()
        input_shape = (height, width, depth)
        chanDim = -1
        
        #if we are using "channels first":
        if K.image_data_format() == "channels_first":
            imput_shape = (depth, height, width)
            chanDim = 1
        #Now add some layers
        model.add(Conv2D(32, (3,3), padding = 'same',
                       input_shape = input_shape))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2,2)))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, (3,3), padding='same'))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(64, (3,3), padding='same'))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2,2)))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(128, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(128, (3, 3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(512))
        model.add(Activation("relu"))
        model.add(BatchNormalization())
        model.add(Dropout(0.25))
        
        #softmax classifier
        model.add(dense(classes))
        model.add(Activation("softmax"))
        
        return model 

In [1]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")
 
# import the necessary packages
from smallvggnet import SmallVGGNet
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import pickle
import cv2
import os

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [14]:
#initialize data and labels
print("[INFO] loading images..")
data = []
labels = []

#grab the image paths and randmly shuffle them
imagePaths = sorted(list(paths.list_images("/Users/trinakarmakar/downloads/keras-tutorial/animals/")))

random.seed(42)
random.shuffle(imagePaths)

#loop over the inout images
for imagePath in imagePaths:
    #load the image resize the inage to be 32x32 pixels (ignoring aspect ratio),
    #flatten the image unto 32X32#3 = 3027 pixel image
    #into a list, and store the image into a data list
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (64,64))
    data.append(image)
    
    #extract the class label from the image and update the 
    #labels list
    label = imagePath.split(os.path.sep)[-2]
    labels.append(label)

data = np.array(data, dtype='float') / 255.0
labels = np.array(labels)

[INFO] loading images..


In [15]:
(X_train, X_test, y_train, y_test) = train_test_split(data, labels, test_size=0.15, random_state=42)

In [16]:
lb = LabelBinarizer()
y_train = lb.fit_transform(y_train)
y_test = lb.transform(y_test)

In [ ]:
aug = ImageDataGenerator(rotation_range=30,
                        width_shift_range=0.1,
                        height_shift_range=0.1,
                        shear_range=0.2,
                        zoom_range=0.2,
                        horizontal_flip = True,
                        fill_mode = "nearest")

#Initialize our VGG like CNN
model = SmallVGGNet.build_model(width=64, height=64, depth=3, 
                                classes=len(lb.classes_))

In [ ]:
# initialize our initial learning rate, # of epochs to train for,
# and batch size
INIT_LR = 0.01
EPOCHS = 75
BS = 32
 
# initialize the model and optimizer (you'll want to use
# binary_crossentropy for 2-class classification)
print("[INFO] training network...")
opt = SGD(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])
 
# train the network
history = model.fit_generator(aug.flow(X_train, y_train, batch_size=BS), 
                              validation_data=(X_test, y_test), 
                              steps_per_epoch=len(X_train) // BS,epochs=EPOCHS)

[INFO] training network...
Epoch 1/75
79/79 [==============================] - 135s 2s/step - loss: 1.2566 - acc: 0.5156 - val_loss: 1.3975 - val_acc: 0.5356
Epoch 2/75
79/79 [==============================] - 130s 2s/step - loss: 0.9586 - acc: 0.6041 - val_loss: 1.0616 - val_acc: 0.5489
Epoch 3/75
79/79 [==============================] - 121s 2s/step - loss: 0.8463 - acc: 0.6146 - val_loss: 1.2536 - val_acc: 0.4778
Epoch 4/75
79/79 [==============================] - 121s 2s/step - loss: 0.7419 - acc: 0.6389 - val_loss: 0.8866 - val_acc: 0.6222
Epoch 5/75
79/79 [==============================] - 121s 2s/step - loss: 0.7334 - acc: 0.6582 - val_loss: 1.2058 - val_acc: 0.5089
Epoch 6/75
79/79 [==============================] - 121s 2s/step - loss: 0.7172 - acc: 0.6607 - val_loss: 0.6663 - val_acc: 0.6778
Epoch 7/75
79/79 [==============================] - 121s 2s/step - loss: 0.6709 - acc: 0.6785 - val_loss: 0.7148 - val_acc: 0.6933
Epoch 8/75
79/79 [==============================] - 4913

79/79 [==============================] - 120s 2s/step - loss: 0.3965 - acc: 0.8252 - val_loss: 0.6388 - val_acc: 0.7444
Epoch 62/75
79/79 [==============================] - 120s 2s/step - loss: 0.4005 - acc: 0.8264 - val_loss: 0.5969 - val_acc: 0.7556
Epoch 63/75
79/79 [==============================] - 120s 2s/step - loss: 0.4110 - acc: 0.8221 - val_loss: 0.5639 - val_acc: 0.7844
Epoch 64/75
79/79 [==============================] - 119s 2s/step - loss: 0.4060 - acc: 0.8243 - val_loss: 0.5370 - val_acc: 0.7733
Epoch 65/75
79/79 [==============================] - 2355s 30s/step - loss: 0.3875 - acc: 0.8284 - val_loss: 0.6751 - val_acc: 0.7533
Epoch 66/75
79/79 [==============================] - 11012s 139s/step - loss: 0.3923 - acc: 0.8252 - val_loss: 0.6118 - val_acc: 0.7644
Epoch 67/75
79/79 [==============================] - 2488s 31s/step - loss: 0.3888 - acc: 0.8240 - val_loss: 0.8795 - val_acc: 0.6956
Epoch 68/75
79/79 [==============================] - 137s 2s/step - loss: 0.4055

In [ ]:
print("[INFO] evaluating network...")
predictions = model.predict(X_test, batch_size=32)
print(classification_report(y_test.argmax(axis=1),
                            predictions.argmax(axis=1), 
                            target_names=lb.classes_))
 
# plot the training loss and accuracy
N = np.arange(0, EPOCHS)
plt.style.use("ggplot")
plt.figure()
plt.plot(N, history.history["loss"], label="train_loss")
plt.plot(N, history.history["val_loss"], label="val_loss")
plt.plot(N, history.history["acc"], label="train_acc")
plt.plot(N, history.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy (SmallVGGNet)")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.savefig("../downloads/acc_report_cnn.png")
 
# save the model and label binarizer to disk
print("[INFO] serializing network and label binarizer...")
model.save("image_classification_cnn.model")
f = open(args["../downloads/image_classification_CNN_lb.pickle", "wb")
f.write(pickle.dumps(lb))
f.close()